In [1]:
import json
from pprint import pp

In [2]:
entries = {};
with open("../output.jsonl") as f:
  for line in f:
    pair = json.loads(line)
    entries[pair[0]] = pair[1]


In [109]:
reflexive_entries = []
for word, entry_list in entries.items():
    if word.endswith('rse'):
        for entry in entry_list:
            if 'form_of' not in entry and entry not in reflexive_entries and entry['pos'] == 'verb' and len(entry['word'].split()) == 1:
                reflexive_entries.append(entry)

reflexive_lemmas = {entry['word'] for entry in reflexive_entries}
print(len(reflexive_entries))
# reflexive_entries
# pp([{"word": entry['word'], "definitions": [entry['definitions']]} for entry in reflexive_entries if not ])
# len([{"word": entry['word'], "definitions": [entry['definitions']]} for entry in reflexive_entries if not all("reflexive" in definition.get("label", "") for definition in entry['definitions'])])
# pp([{"word": entry['word'], "definitions": [entry['definitions']]} for entry in reflexive_entries if not all("reflexive" in definition.get("label", "") for definition in entry['definitions'])])

247


In [56]:
has_clean_non_combined_form_reflexive_entry = []
has_problematic_non_combined_form_reflexive_entry = []
does_not_have_non_combined_lemma_entry = []
for reflexive_entry in reflexive_entries:
  non_combined_lemma = reflexive_entry.get("word", "")[:-2]
  if entries.get(non_combined_lemma):
    non_combined_entry = entries.get(non_combined_lemma)
    triplet = (
      len(non_combined_entry) == 1,
      len(non_combined_entry[0]['definitions']) == 1,
      "only used in" in non_combined_entry[0]['definitions'][0]['definition']
    )
    if all(triplet):
      # print("in first condition?")
      has_clean_non_combined_form_reflexive_entry.append((reflexive_entry, triplet))
    else:
      # print("in else?", non_combined_entry, non_combined_entry[0]['definitions'][0])
      has_problematic_non_combined_form_reflexive_entry.append((reflexive_entry, non_combined_entry, triplet))
  else:
    does_not_have_non_combined_lemma_entry.append(reflexive_entry)


# print(len([entry for entry in [(entry, entries.get(entry['word'][:-2]), len(entries.get(entry['word'][:-2])) == 1, len(entries.get(entry['word'][:-2])[0]['definitions']) == 1, "only used in" in entries.get(entry['word'][:-2])[0]['definitions'][0]['definition']) for entry in reflexive_entries if entries.get(entry['word'][:-2])] 
#  if any([not x for x in entry[2:]])]))
# pp(([entry for entry in [(entry, entries.get(entry['word'][:-2]), len(entries.get(entry['word'][:-2])) == 1, len(entries.get(entry['word'][:-2])[0]['definitions']) == 1, "only used in" in entries.get(entry['word'][:-2])[0]['definitions'][0]['definition']) for entry in reflexive_entries if entries.get(entry['word'][:-2])] 
#  if any([not x for x in entry[2:]])])[0])

# print(len([entry for entry in [(entry, entries.get(entry['word'][:-2]), len(entries.get(entry['word'][:-2])) == 1, len(entries.get(entry['word'][:-2])[0]['definitions']) == 1, "only used in" in entries.get(entry['word'][:-2])[0]['definitions'][0]['definition']) for entry in reflexive_entries if entries.get(entry['word'][:-2])] 
#  if not any([not x for x in entry[2:]])]))

print("clean:", len(has_clean_non_combined_form_reflexive_entry))
print("problematic:", len(has_problematic_non_combined_form_reflexive_entry))
pp([entry, has_problematic_non_combined_form_reflexive_entry])

clean: 206
problematic: 1
[{'word': 'baladronearse',
  'pos': 'verb',
  'from_forms': True,
  'form_of': 'baladronear',
  'definitions': []},
 [({'word': 'incautarse',
    'pos': 'verb',
    'forms': ['me incauté',
              'incautarme',
              'incautarte',
              'te incautarías',
              'incautaros',
              'incáutese',
              'os incautabais',
              'se incautará',
              'te incautaste',
              'se incautare',
              'se incauten',
              'te incautares',
              'se incautarían',
              'os incautareis',
              'incáutate',
              'os incautarais',
              'se incautasen',
              'incautándonos',
              'te incautes',
              'nos incautáremos',
              'se incautaba',
              'me incautaría',
              'incautándote',
              'os incautéis',
              'incautarse',
              'te incautarás',
              'se incautaran',


so, for 206 (of 247 total reflexive), the non-combined form of the lemma just points right back at the reflexive lemma, 

(and for 1 more it's slightly fussier but we could still trivially wipe out the entry for the non-combined lemma (maybe hard to identify algorithmically))

In [59]:
len(does_not_have_non_combined_lemma_entry)
does_not_have_non_combined_lemma_entry

[{'word': 'petatearse',
  'pos': 'verb',
  'forms': ['me petateé', 'petateado', 'me petateo'],
  'definitions': [{'label': 'colloquial, Chile, El Savador, Mexico, Nicaragua, Panama, takes a reflexive pronoun',
    'definition': 'to die'}]},
 {'word': 'agolparse',
  'pos': 'verb',
  'forms': ['agolpándote',
   'se agolparía',
   'os agolparais',
   'os agolparéis',
   'se agolparán',
   'agolpándome',
   'me agolparía',
   'os agolpaseis',
   'se agolpan',
   'se agolpara',
   'se agolpe',
   'agolpaos',
   'me agolpara',
   'agolparme',
   'agolpada',
   'os agolpáis',
   'os agolpareis',
   'te agolparías',
   'agolparnos',
   'agolparos',
   'nos agolparemos',
   'te agolparas',
   'se agolpa',
   'agolpándose',
   'me agolpare',
   'te agolpabas',
   'agolparte',
   'nos agolpábamos',
   'se agolpó',
   'se agolpare',
   'agólpate',
   'os agolparíais',
   'os agolpasteis',
   'se agolpará',
   'os agolpéis',
   'me agolpaba',
   'me agolpase',
   'nos agolpáramos',
   'agolparse',


In [116]:
def definition_does_have_reflexive_label(definition):
  return "reflexive" in definition.get("label", "")
  print(all(["reflexive" in definition.get("label", "") for definition in entry['definitions']]))

def definition_does_have_reflexive_in_definition(definition):
  return "reflexive" in definition.get("definition", "")

def entry_does_have_reflexive_label_on_some_definition(entry):
  return any(definition_does_have_reflexive_label(definition) or definition_does_have_reflexive_in_definition(definition) for definition in entry['definitions'])

In [119]:
does_not_have_reflexive_label = [entry for entry in reflexive_entries if not entry_does_have_reflexive_label_on_some_definition(entry)]
print(len(does_not_have_reflexive_label))
has_reflexive_on_definition = [entry for entry in reflexive_entries if any(definition_does_have_reflexive_in_definition(definition) for definition in entry['definitions'])]
print(len(has_reflexive_on_definition))
pp(has_reflexive_on_definition)

82
1
[{'word': 'enchicharse',
  'pos': 'verb',
  'forms': ['enchichado', 'me enchicho', 'me enchiché'],
  'definitions': [{'label': 'Latin America',
                   'definition': '(reflexive of enchichar) to get drunk on '
                                 'chicha'},
                  {'label': 'Andes, Colombia, Central America',
                   'definition': 'to become angry'}]}]


In [104]:
does_not_have_reflexive_label

[{'word': 'ufanarse',
  'pos': 'verb',
  'forms': ['se ufanase',
   'te ufanaste',
   'se ufanare',
   'ufanadas',
   'ufanarnos',
   'te ufanes',
   'os ufanabais',
   'me ufane',
   'os ufanáis',
   'me ufanaba',
   'ufanada',
   'se ufanará',
   'se ufanaba',
   'te ufanarás',
   'ufanándome',
   'se ufanaran',
   'ufanaos',
   'ufanados',
   'nos ufanáremos',
   'ufánese',
   'me ufanaré',
   'ufanándose',
   'ufanándonos',
   'se ufana',
   'te ufanarías',
   'ufánate',
   'te ufanaras',
   'nos ufanábamos',
   'os ufanaréis',
   'se ufanaría',
   'os ufanaríais',
   'te ufanabas',
   'se ufanasen',
   'nos ufanemos',
   'ufanándoos',
   'se ufanan',
   'nos ufanaríamos',
   'ufanaros',
   'me ufané',
   'se ufanaren',
   'ufanate',
   'ufanémonos',
   'os ufanarais',
   'ufanarse',
   'te ufanás',
   'nos ufanamos',
   'os ufanasteis',
   'se ufane',
   'os ufanéis',
   'ufanarme',
   'se ufanó',
   'ufanarte',
   'me ufanaría',
   'ufánense',
   'se ufanarán',
   'se ufanen',
  

In [124]:
reflexive_lemmas_with_forms_pointing_to_them = set()
form_of_points_to_reflexive = []
for word_entry in entries.values():
  for entry in word_entry:
    if entry.get("form_of") in reflexive_lemmas:
      form_of_points_to_reflexive.append(entry)
      reflexive_lemmas_with_forms_pointing_to_them.add(entry.get("form_of"))

In [129]:
print(len(form_of_points_to_reflexive))
print(len(reflexive_lemmas_with_forms_pointing_to_them))
print(reflexive_lemmas_with_forms_pointing_to_them == reflexive_lemmas)


27325
247
True


In [130]:
pp(form_of_points_to_reflexive[:10])

[{'word': 'muse',
  'pos': 'verb',
  'form_of': 'musirse',
  'definitions': [{'definition': 'only used in se muse, third-person singular '
                                 'present indicative of musirse'},
                  {'definition': 'only used in te ... muse, syntactic variant '
                                 'of músete, second-person singular imperative '
                                 'of musirse'}]},
 {'word': 'jambe',
  'pos': 'verb',
  'form_of': 'jambarse',
  'definitions': [{'definition': 'only used in me jambe, first-person singular '
                                 'present subjunctive of jambarse'},
                  {'definition': 'only used in se jambe, third-person singular '
                                 'present subjunctive of jambarse'},
                  {'definition': 'only used in se ... jambe, syntactic variant '
                                 'of jámbese, third-person singular imperative '
                                 'of jambarse'}]},
 {'word':